In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import os

from functions.utils import state_dict_to_vector

from model.models_kde import TracksToKDE_Ellipsoids_DDplus as Model
from model.kde_loss_Ba import Loss
from model.collectdata_kde_Ellipsoids import collect_t2kde_data
device = 'cuda:0'

In [2]:
#dict with run_title : (num_events (in thousands), list of (num_epochs, dir_name))
all_runs = {
    'Adam 18k':(18,
                      [(200, 'welltrained_DDplus_Adamcontrol')]),
    'Adam 80k':(80,
                      [(50, 'welltrained_DDplus_Adam_80k')]),
    'Adam 80k higher lr':(80,
                      [(50, 'welltrained_DDplus_Adam_80k_higherlr')]),
    'Adam 80k lower lr':(80,
                      [(50, 'welltrained_DDplus_Adam_80k_lowerlr')]),
    'adaptive careful Adam 80k':(80,
                      [(50, 'welltrained_DDplus_Adam_80k_adaptiveAdam')]),
    'Adam EVE 80k':(80,
                      [(108, 'welltrained_DDplus_80k_Adam_EVE')]), 
    }

#to quickly call runs via number
run_names = {}
for i,key in enumerate(list(all_runs)):
    print(i, ':', key)
    run_names[i] = key

0 : Adam 18k
1 : Adam 80k
2 : Adam 80k higher lr
3 : Adam 80k lower lr
4 : adaptive careful Adam 80k
5 : Adam EVE 80k


In [3]:
#Select runs to plot
runs = [i for i in range(len(all_runs))]

run_labels = [run_names[i] for i in runs]

#now to generate list of paths to the stored model dicts
dict_paths = []
num_epochs = []
for run_label in run_labels:
    these_paths = []
    this_num_epochs = 0
    dir_tuples = all_runs[run_label][1]
    for dir_tuple in dir_tuples:
        
        this_num_epochs += dir_tuple[0]
        dir_name = dir_tuple[1]
        for j in range(dir_tuple[0]):
            #print('ML/' + dir_names[i] + '/' + dir_names[i] + '_%i.pyt'%(j))
            these_paths.append('baileyds/DDplus/' + dir_name + '/' + dir_name + '_%i.pyt'%(j))
    dict_paths.append(these_paths)
    num_epochs.append(this_num_epochs)   
    
print(dict_paths)

[['baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_0.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_1.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_2.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_3.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_4.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_5.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_6.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_7.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_8.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_9.pyt', 'baileyds/DDplus/welltrained_DDplus_Adamcontrol/welltrained_DDplus_Adamcontrol_10.pyt', 'baileyds/DDplus/welltrained_DDplus_Adam

In [4]:
for paths in dict_paths:
    for filepath in paths:
        if os.path.exists(filepath) == False:
            print(filepath, os.path.exists(filepath))

In [5]:
## Training dataset. You can put as many files here as desired.

##train_loader = collect_t2kde_data('/share/lazy/pv-finder/20k_evts_for_KDE_learning_200716.h5',
# train_data = collect_t2kde_data('/share/lazy/will/data/June30_2020_80k_1.h5', 
#                                   #'/share/lazy/will/data/June30_2020_80k_2.h5',
#                                   #'/share/lazy/will/data/June30_2020_80k_3.h5',
#                                   #'/share/lazy/will/data/June30_2020_80k_4.h5',
#                              batch_size=32,
# ## if we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
# ## the dataset will overflow the GPU memory; device=device will allow the data to move back
# ## and forth between the CPU and GPwU memory. While this allows use of a larger dataset, it slows
# ## down performance by about 10%.  So comment out when not needed.
#                              device=device,
#                            slice = slice(None,1000)
#                            )

# Validation dataset. You can slice to reduce the size.
## mds no separate validation set yet,

# For iter12, change slice(18000,None) to slice(10000,None)
# First, we'll see if this changes the validation cost significantly
#  Second, we will see if this reduces the validation cost fluctuations
train_data = collect_t2kde_data('/share/lazy/sokoloff/ML-data_AA/20K_POCA_kernel_evts_200926.h5',
                            batch_size=64,
                            device=device,
                            slice = slice(19000,None)
                           )

Loading data...
pocaMx.shape =  (20000,)
nEvts =  20000
len(pocaMx[0]) =  211
len(pocaMx[1]) =  21
len(pocaMx[2]) =  20
len(pocaMx[3]) =  198
len(pocaMx[4]) =  233
majorAxis.shape =  (20000, 3)
minorAxis_1.shape =  (20000, 3)
minorAxis_2.shape =  (20000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  20000
 iEvt, nTrks =  0 211
 iEvt, nTrks =  1 21
 iEvt, nTrks =  2 20
 iEvt, nTrks =  3 198
 iEvt, nTrks =  4 233
 iEvt, nTrks =  5 85
 iEvt, nTrks =  6 223
 iEvt, nTrks =  7 425
 iEvt, nTrks =  8 252
 iEvt, nTrks =  9 169
A.shape =  (20000,)
majorAxis[iTrk][0][0] =  -0.00023452607
majorAxis[iTrk][1][0] =  -0.00047206535
majorAxis[iTrk][2][0] =  0.096502915
minorAxis_1[iTrk][0][0] =  -15.822749
minorAxis_1[iTrk][1][0] =  7.8608756
minorAxis_1[iTrk][2][0] =  -2.6228399e-08
minorAxis_2[iTrk][0][0] =  7.860759
minorAxis_2[iTrk][1][0] =  15.822513
minorAxis_2[iTrk][2][0] =  0.096502915
  
majorAxis[iTrk][0][0] =  -0.37655562
majorAxis[iTrk][1][0] =  0.2768704
majorAxis[iTrk][2][0] = 

In [6]:
nOut1 = 50
nOut2 = 50
nOut3 = 50
nOut4 = 50
nOut5 = 50
nOut6 = 50
nOut7 = 50
nOut8 = 50
nOut9 = 50
nOut10 = 50
nOut11 = 50
latentChannels = 4
model = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11,latentChannels)
model.to(device)
loss_fn = Loss(epsilon=3e-5)

In [7]:
def train_loss(dataloader, model, loss_fn):
    model.train()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    return test_loss/len(dataloader)

def eval_loss(dataloader, model, loss_fn):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    return test_loss/len(dataloader)

In [ ]:
#train_losses = []
#saving the loss data so I don't have to recalculate each time

eval_losses = []

scan_sizes = {18:1, 80:1, 160:3, 320:1}

for i,paths in enumerate(dict_paths):
    print('On run:', run_labels[i])
    these_train_losses = []
    these_eval_losses = []
    scan_size = scan_sizes[all_runs[run_labels[i]][0]]
    print(f'{all_runs[run_labels[i]][0]}k events -> scan size = {scan_size}')
    for ep in np.arange(0, num_epochs[i], scan_size):
        print("Epoch:", ep, '/', num_epochs[i])
        state_dict = torch.load(paths[ep], map_location=device)
        model.load_state_dict(state_dict)
#         start_time = time.time()
#         these_train_losses.append(train_loss(train_data, model, loss_fn))
#         print("Train loss time:", time.time()-start_time)
        start_time = time.time()
        these_eval_losses.append(eval_loss(train_data, model, loss_fn))
        print("Eval loss time:", time.time()-start_time)
        print('#############################################')
    #eval_losses.append(these_eval_losses)
    #add the last values if not added
    if ep != (len(paths)-1):
        ep = (len(paths)-1)
        print("Epoch:", ep, '/', num_epochs[i])
        state_dict = torch.load(paths[ep], map_location=device)
        model.load_state_dict(state_dict)
#         start_time = time.time()
#         these_train_losses.append(train_loss(train_data, model, loss_fn))
#         print("Train loss time:", time.time()-start_time)
        start_time = time.time()
        these_eval_losses.append(eval_loss(train_data, model, loss_fn))
        print("Eval loss time:", time.time()-start_time)
        print('#############################################')
    eval_losses.append(these_eval_losses)
    
with open('welltrained_analysis_savedloss.txt', 'w') as f:
    for i in range(len(dict_paths)):
        print(i)
        this_loss = eval_losses[i]
        for j in range(len(this_loss)):
            f.write(str(this_loss[j]))
            if j+1 in range(len(this_loss)):
                f.write(',')
        if i+1 in range(len(dict_paths)):
            f.write('\n')

On run: Adam 18k
18k events -> scan size = 1
Epoch: 0 / 200
Eval loss time: 0.6834151744842529
#############################################
Epoch: 1 / 200
Eval loss time: 0.6330766677856445
#############################################
Epoch: 2 / 200
Eval loss time: 0.6240203380584717
#############################################
Epoch: 3 / 200
Eval loss time: 0.6228663921356201
#############################################
Epoch: 4 / 200
Eval loss time: 0.6275627613067627
#############################################
Epoch: 5 / 200
Eval loss time: 0.6249222755432129
#############################################
Epoch: 6 / 200
Eval loss time: 0.62546706199646
#############################################
Epoch: 7 / 200
Eval loss time: 0.6265971660614014
#############################################
Epoch: 8 / 200
Eval loss time: 0.6095185279846191
#############################################
Epoch: 9 / 200
Eval loss time: 0.6237542629241943
###########################################

Eval loss time: 0.6392052173614502
#############################################
Epoch: 86 / 200
Eval loss time: 0.6351327896118164
#############################################
Epoch: 87 / 200
Eval loss time: 0.6394319534301758
#############################################
Epoch: 88 / 200
Eval loss time: 0.6201338768005371
#############################################
Epoch: 89 / 200
Eval loss time: 0.6052169799804688
#############################################
Epoch: 90 / 200
Eval loss time: 0.6115005016326904
#############################################
Epoch: 91 / 200
Eval loss time: 0.6237742900848389
#############################################
Epoch: 92 / 200
Eval loss time: 0.6183910369873047
#############################################
Epoch: 93 / 200
Eval loss time: 0.6266593933105469
#############################################
Epoch: 94 / 200
Eval loss time: 0.622063159942627
#############################################
Epoch: 95 / 200
Eval loss time: 0.6238658428192

In [ ]:
eval_losses = []
with open('welltrained_analysis_savedloss.txt', 'r') as f:
    idx = 0
    for line in f:
        line.split(',')
        eval_losses.append([float(val) for val in line.split(',')])
        print(eval_losses)

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(len(dict_paths)):
    scan_size = scan_sizes[all_runs[run_labels[i]][0]] 
    #print(scan_size)
    if np.arange(0, num_epochs[i], scan_size)[-1] != (num_epochs[i]-1):
        plt.plot(np.append(np.arange(0, num_epochs[i], scan_size), [num_epochs[i]-1]), eval_losses[i], 'o-', label=run_labels[i])
    else:
        plt.plot(np.arange(0, num_epochs[i], scan_size), eval_losses[i], 'o-', label=run_labels[i])
plt.xlabel("Epoch")
plt.ylabel("Val. Loss")
plt.yscale('log')
plt.legend()

#now approx. normalizing x axis for comp time
plt.figure(figsize=(12,8))
for i in range(len(dict_paths)):
    num_evts = all_runs[run_labels[i]][0]
    scan_size = scan_sizes[all_runs[run_labels[i]][0]] 
    #print(scan_size)
    if np.arange(0, num_epochs[i], scan_size)[-1] != num_epochs[i]-1:
        plt.plot(num_evts*np.append(np.arange(0, num_epochs[i], scan_size), [num_epochs[i]-1]), eval_losses[i], 'o-', label=run_labels[i])
    else:
        plt.plot(num_evts*np.arange(0, num_epochs[i], scan_size), eval_losses[i], 'o-', label=run_labels[i])
plt.xlabel("Num. kiloevts*epochs")
plt.ylabel("Val. Loss")
plt.yscale('log')
plt.legend()

In [ ]:
#epoch_steps = [param_states[i] - param_states[i-1] for i in range(1, len(param_states))]

def angle_between(i, j, epoch_steps):
    if i == j:
        return 0
    else:
        #print(np.dot(epoch_steps[i], epoch_steps[j])/(np.linalg.norm(epoch_steps[i])*np.linalg.norm(epoch_steps[j])))
        return np.arccos(np.dot(epoch_steps[i], epoch_steps[j])/(np.linalg.norm(epoch_steps[i])*np.linalg.norm(epoch_steps[j])))

def angle_between(a, b):
    #print(np.dot(a, b), np.linalg.norm(a)*np.linalg.norm(b))
    return np.arccos( np.dot(a, b) /(np.linalg.norm(a)*np.linalg.norm(b)) )
    
#num_steps = num_epochs-1
# angle_grid = np.zeros((num_steps, num_steps))
# for i in range(num_steps):
#     for j in range(i+1, num_steps):
#         print(i, j)
#         angle = angle_between(i, j, epoch_steps)
#         angle_grid[i, j] = angle
#         angle_grid[j, i] = angle

angle_grids = []
grid_size = 25

ep_intervals = [(np.arange(0, total_epochs+1, grid_size)[i], np.arange(0, total_epochs+1, grid_size)[i+1]) for i in range(len(np.arange(0, total_epochs+1, grid_size))-1)]
print(ep_intervals)
for interval in ep_intervals:
    
    these_param_states = []
    for i in range(interval[0], interval[1]):
        if i % 100 == 0:
            print(i)
        these_param_states.append(state_dict_to_vector(torch.load(file_names[i], map_location='cpu')))
        #print("Loaded", file_names[i])
#         if i >= 0 and i < num_epochs:
#             #print('a')
#             these_param_states.append(state_dict_to_vector(torch.load('ML/'+dir_name+'/'+dir_name+'_%i.pyt'%(i), map_location='cpu')))
#             print("Loaded", 'ML/'+dir_name+'/'+dir_name+'_%i.pyt'%(i))
#         if i >= num_epochs and i < num_epochs2:
#             #print('b')
#             these_param_states.append(state_dict_to_vector(torch.load('ML/'+dir_name2+'/'+dir_name2+'_%i.pyt'%(i-6), map_location='cpu')))
#             print("Loaded", 'ML/'+dir_name2+'/'+dir_name2+'_%i.pyt'%(i-6))
        
    interval_size = len(these_param_states)-1    
    these_steps = [these_param_states[i+1]-these_param_states[i] for i in range(interval_size)]
    
    this_angle_grid = np.zeros((interval_size, interval_size))
    
    for i in range(interval_size):
        for j in range(i+1, interval_size):#range(i+1, interval_size):
            angle = angle_between(these_steps[i], these_steps[j])
            #print(angle)
            this_angle_grid[i,j] = angle
            this_angle_grid[j,i] = angle
            
    angle_grids.append(this_angle_grid)
    
#     end = state_dict_to_vector(torch.load('ML/'+dir_name+'/'+dir_name+'_%i.pyt'%(i+1)))
#     beginning = state_dict_to_vector(torch.load('ML/'+dir_name+'/'+dir_name+'_%i.pyt'%(i)))
#     a = end-beginning
#     for j in np.arange(i+1, num_steps, 10):
#         print(i, j)
#         end2 = state_dict_to_vector(torch.load('ML/'+dir_name+'/'+dir_name+'_%i.pyt'%(j+1)))
#         beginning2 = state_dict_to_vector(torch.load('ML/'+dir_name+'/'+dir_name+'_%i.pyt'%(j)))
#         b = end2-beginning2
#         angle = angle_between(a, b)
        
#         angle_grid[i, j] = angle
#         angle_grid[j, i] = angle

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cm

#print(angle_grids)
for angle_grid in angle_grids:
    plt.figure(figsize=(12,8))
    norm = colors.Normalize(vmin=np.amin(angle_grid), vmax=np.amax(angle_grid))   
    plt.imshow(angle_grid, cmap=cm.jet)
    plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cm.jet))
    plt.xlabel('Step i')
    plt.ylabel('Step j')
    
#print("num params:", len(these_param_states[0]))

In [ ]:
step_sizes = []
beg = state_dict_to_vector(torch.load(file_names[0], map_location='cpu'))
for i in range(total_epochs-1):
    if i % 100 == 0:
        print(i)
    end = state_dict_to_vector(torch.load(file_names[i+1], map_location='cpu'))
    step_sizes.append(np.linalg.norm(end-beg))
    beg = end

#plotting size of epoch step
plt.figure(figsize=(12,8))
plt.title("Size of epoch steps")
plt.xlabel("Epoch step")
plt.ylabel("Norm. of step")
plt.yscale('log')
plt.plot(step_sizes, 'o-')

In [ ]:
from functions.plotting_DDplus_hists import load_data

state_dicts = []
for i in np.arange(0, total_epochs, 25):
    state_dicts.append( (i, torch.load(file_names[i], map_location='cpu')) )

data = load_data()

In [ ]:
from functions.plotting_DDplus_hists import plot_DDplus_hists as Plot
Plot(state_dicts, data, num_events=3)